In [1]:
!nvidia-smi

Thu May 16 09:22:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# !pip install transformers[sentencepiece]
# !pip install datasets
# !pip install sacrebleu rouge_score
# !pip install py7zr # For unziping the file

!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q # -q means quite mode.
!pip install --upgrade accelerate # accelerate is for GPU to run distributed processes.
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 

- `Sacrebleu` is a metric commonly used for evaluating the quality of machine-generated translations, typicaly in the context of machine translation tasks. It often used in NLP competions and research to access the performace of translation model.

- Rouge (Recall-Oriented Understudy for Gisting Evaluation) is a metrics used for evaluating the quality of summaries by comparing them to reference summaries. Rouge metrics are commonly used in NLP processing tasks, particularly in text summarization systems.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk, torch, tqdm # tqdm is used to get the process work
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from datasets import load_dataset, load_from_disk, load_metric

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
device = 'cuda' if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# Using the Pre-trained Pegasus Model with 2Billion Parameters
model = 'google/pegasus-cnn_dailymail'
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [6]:
# Loading the Pegasus Model.
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

## Different Approaches.
- Approach 1. Fine-Tuning of Pre-Trained Model or
- Approach 2. Create A Knowledge Base and Extract the information from the Knowledge Base and Connect it through Query to generate the Output.
This process is called as RAG i.e. Retrieval Augmented Generation.

## Fine-Tuning of Pre-Trained Model.
- Step 1: Training of the model
- Step 2: Inferencing of the model i.e. loading the pretrained model and predicting the results.

In [8]:
# Loading the In-built Dataset.
dataset = load_dataset('samsum')
dataset

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
dataset['train']

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [10]:
print(dataset['train']['dialogue'][0])
print('\n')
print(dataset['train']['dialogue'][1])

Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)


Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great


In [11]:
for data in dataset:
  print(dataset[data])

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 818
})


In [12]:
dataset['train'].column_names

['id', 'dialogue', 'summary']

In [13]:
# Tokenization
def examples_to_features(data_in_batch):
  input_encoding = tokenizer(data_in_batch['dialogue'], max_length = 1024, truncation = True) # Truncation means cutting of sentences.
  target_encoding = tokenizer(data_in_batch['summary'], max_length = 128, truncation = True)
  return {
      "input_ids": input_encoding['input_ids'],
      "attention_mask": input_encoding['attention_mask'],
      "labels": target_encoding['input_ids']
  }

In [14]:
dataset_en = dataset.map(examples_to_features, batched = True)
dataset_en

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [15]:
dataset_en['train']['input_ids'][0] # To check the encoded train data.

[12195,
 151,
 125,
 7091,
 3659,
 107,
 842,
 119,
 245,
 181,
 152,
 10508,
 151,
 7435,
 147,
 12195,
 151,
 125,
 131,
 267,
 650,
 119,
 3469,
 29344,
 1]

- In the context of NLP, an attention mask is a `binary mask`(0 or1) that indicates which tokens in a sequence should be attended to and which to be ignored.
- The mask is used during the self-attention mechanism to control the flow of information between different positions in the input sequence.
- 1: Indicates that the corresponding token should be `ATTENDED` (included in the computation of the weights.)
- 0: Indicates that the corresponding token should be `IGNORED`(excluded in the computation of the weights.)
- For Eg: tokens = ['how', 'are', '', 'you']. For this attention mask would be [1,1,1,1]. In this case, all tokens are marked with 1, indicating that each token should be attended to during the self-attention process.
- However, in situations where padding is used to make sequences of equal length (common in batch processing), the attention mask may be adjusted to indicate which positions correspond to actual tokens and which correspond to padding. For example: [1,1,1,1,0] Here, the last token is marked as 0, indicating that it corresponds to padded token and should be ignored during attention.

In [16]:
# To check the attention mask of train data.
dataset_en['train']['attention_mask'][0]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [17]:
# Implementing Fine-Tuning and RAG will be considered in later stages for other projects.
from transformers import TrainingArguments, Trainer

In [18]:
training_args = TrainingArguments(
    output_dir = 'pegasus-samsum',
    num_train_epochs = 1,
    warmup_steps = 500,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size= 1,
    weight_decay = 0.01,
    logging_steps = 10,
    evaluation_strategy = 'steps',
    eval_steps = 500,
    save_steps = 1e6,
    gradient_accumulation_steps = 16
)
training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=500,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla

In [19]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [20]:
trainer = Trainer(model = model_pegasus, args = training_args, tokenizer = tokenizer, data_collator = seq2seq_data_collator,
                  train_dataset =dataset_en['train'], eval_dataset = dataset_en['validation'] )
trainer

- https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments
- https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.Seq2SeqTrainer

In [21]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0043694084765864, metrics={'train_runtime': 162.32, 'train_samples_per_second': 5.046, 'train_steps_per_second': 0.314, 'total_flos': 313450454089728.0, 'train_loss': 3.0043694084765864, 'epoch': 0.9963369963369964})

In [22]:
# To Save the Fine-tuned Model.
model_pegasus.save_pretrained('pegasus_samsum_model')
tokenizer.save_pretrained('samsum_tokenizer')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


('samsum_tokenizer/tokenizer_config.json',
 'samsum_tokenizer/special_tokens_map.json',
 'samsum_tokenizer/spiece.model',
 'samsum_tokenizer/added_tokens.json',
 'samsum_tokenizer/tokenizer.json')

In [24]:
# Loading Our Saved Model.
tokenizer = AutoTokenizer.from_pretrained('samsum_tokenizer')
model = AutoModelForSeq2SeqLM.from_pretrained('pegasus_samsum_model')

In [25]:
# Predictions From Saved Model.
test_data = dataset_en['test']
test_data

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 819
})

In [26]:
sample_dialogue = dataset_en['test']['dialogue'][1]
print(sample_dialogue)

Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)


In [28]:
sample_summary = dataset_en['test']['summary'][0]
print(sample_summary)

Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [29]:
pipeline_obj = pipeline('summarization', model = model, tokenizer = tokenizer)
pipeline_obj

In [30]:
# This pipeline runs for complete data
pipeline_obj(sample_dialogue)

[{'summary_text': "Eric: Machine! Rob: Machine! Eric: Machine! Rob: TTYL?<n>Rob: Sure :) Eric: Gr8! I'll watch them now! Rob: Me too! Eric: Machine! Rob: Machine! Eric: TTYL? Rob: Sure :)"}]

In [31]:
# Fetching only Summary from the Pipeline.
prediction = pipeline_obj(sample_dialogue)[0]['summary_text']

In [32]:
# Actual Summary
sample_summary

"Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."

In [34]:
# Model Evaluation Custom Function.
def generate_batch_sized_chunks(list_of_elements, batch_size):
    ''' Split the dataset into smaller batches that we can process simultaneously
      Yeild successive batch-sized chunks from list_of_elements.'''
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i+batch_size]

def calculate_metrics_on_test_ds(dataset, metric, model, tokenizer, batch_size = 16,
                                 device = device, column_text = 'article',column_summary = 'highlights'):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size = batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size = batch_size))
    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total = len(article_batches)):
        inputs = tokenizer(article_batch, max_length = 1024, truncation = True,
                       padding = 'max_length', return_tensors = 'pt')
        summaries = model.generate(input_ids = inputs['input_ids'].to(device),
                               attention_mask = inputs['attention_mask'].to(device),
                               length_penalty = 0.8, num_beams = 8, max_length =128)
        ''' Parameter for length penalty ensures that the model does not generate sequence '''
        decoded_summaries = [tokenizer.decode(s,skip_special_tokens = True, clean_up_tokenization_spaces = True) for s in summaries]
        decoded_summaries = [d.replace(""," ") for d in decoded_summaries]
        metric.add_batch(predictions = decoded_summaries, references = target_batch)
    score = metric.compute() # Computes and returns the ROUGE Score.
    return score


In [37]:
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_metric = load_metric('rouge')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [38]:
score = calculate_metrics_on_test_ds(dataset_en['test'][0:10], rouge_metric, trainer.model, tokenizer,
                                     batch_size = 2, column_text = 'dialogue', column_summary = 'summary' )
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = [f'pegasus'])

100%|██████████| 5/5 [00:14<00:00,  2.90s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.083982,0.005399,0.075415,0.084064


- `rouge1`: Unigram(1-gram) based scoring
- `rouge2`: Bigram(2-gram) based scoring
- `rougeL`: Longest Common subsequence based scoring
- `rougeLSum`: Splits text using '\n'